# Dubbing Task

This is an example Python notebook for running dubbing tasks using [Camb.ai](https://camb.ai)'s API

# Index
- [Optional: Loading API Key from .env file](#optional-loading-api-key-from-env-file)
- [Dub Method](#dub-method)
- [Individual Methods](#individual-methods)
  - [Create End-to-End Dubbing Task](#create-end-to-end-dubbing-task)
  - [Get Dubbing Status](#get-dubbing-status)
  - [Get Dubbed Run Info](#get-dubbed-run-info)

In [ ]:
# !pip install cambai

To get started, make sure you import both the `CambAI` class from the `cambai` module.

Ensure that you have the Camb AI API Key from https://studio.camb.ai

In [ ]:
import os
from cambai import CambAI

In [ ]:
client = CambAI(
    api_key=os.environ.get("CAMB_API_KEY") # Get your API key from https://studio.camb.ai
)

### Optional: Loading API Key from .env file

While you can provide an `api_key` as an argument, we recommend using [`python-dotenv`](https://pypi.org/project/python-dotenv/) to add `CAMB_API_KEY="Your API Key"` to your `.env` file to prevent your API Key from being stored in source control.

In [ ]:
# !pip install python-dotenv

In [ ]:
from dotenv import load_dotenv

load_dotenv()

client = CambAI(
    api_key=os.environ.get("CAMB_API_KEY") # Get your API key from https://studio.camb.ai
)

----

To get the numbers for the list of all available languages, you can use the `get_languages()` method.

You can also save the list of languages to a file by setting the `write_to_file` argument to `True`.

In [ ]:
client.get_languages("source", write_to_file=True)
# client.get_languages("target", write_to_file=True)

---
---

## Dub Method

You can call the `dub` method to perform the entire end-to-end dubbing process.<br>
From calling the API, to getting the final dubbed run info.

In [ ]:
values = client.dub(
    video_url="https://youtu.be/rlK-BySadHk", # Any publicly accessible video URL
    source_language=37,  # English (Ireland)
    target_language=54,  # Spanish (Spain)
    debug=True,          # You can set this to True to see the progress of the dubbing process
    polling_interval=70
)

**NOTE:** The dubbing process can take a while to complete, so it's better to set the polling interval to a higher value, especially for longer videos.

In [ ]:
values["video_url"]

In [ ]:
values["audio_url"]

### Optional: Downloading via [`yt-dlp`](https://pypi.org/project/yt-dlp/)

You can also download the dubbed video using [`yt-dlp`](https://pypi.org/project/yt-dlp/).

In [ ]:
# !pip install yt-dlp

In [ ]:
from yt_dlp import YoutubeDL

values = client.dub(
    video_url="https://youtu.be/rlK-BySadHk", # Any publicly accessible video URL
    source_language=37,  # English (Ireland)
    target_language=54,  # Spanish (Spain)
    debug=True,          # You can set this to True to see the progress of the dubbing process
    polling_interval=70
)

URLS = [
    values["video_url"],
    values["audio_url"]
]

ydl_opts = {
    "outtmpl": f"dubbed_video_{values["run_id"]}" + ".%(ext)s",
    "format": "mp4", # Change this to "mp3" if you want to download the audio only
    "overwrite": True,
}

with YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(URLS)

---
---

## Individual Methods

You can also access individual methods to have more control over the dubbing task.

- [`start_dubbbing()`](#create-end-to-end-dubbing-task)
- [`get_dubbing_status()`](#get-dubbing-status)
- [`get_dubbed_run_info()`](#get-dubbed-run-info)

### Create End-to-End Dubbing Task

This method creates the dubbing task and returns the `run_id` which can be used to check the status of the dubbing task.

In [ ]:
task = client.start_dubbing("https://youtu.be/rlK-BySadHk", 37, 125)

task_id = task["task_id"]

In [ ]:
task_id = '31f7af5f-346d-49c9-930b-f2d5e5dc5b4a'

----

### Get Dubbing Status

This method returns the status of the dubbing task.

If you got the "SUCCESS" status, then the dubbing task is completed and you can get the download link for the dubbed audio from `get_dubbed_run_info()`.

The status can be one of the following:
- SUCCESS
- PENDING
- TIMEOUT
- ERROR
- PAYMENT_REQUIRED

In [ ]:
dubbing_status = client.get_task_status("dubbing", task_id)
# dubbing_status = client.get_dubbing_status(task_id)

status = dubbing_status["status"]
run_id = dubbing_status["run_id"]

In [ ]:
status, run_id

**NOTE:** The dubbing task may take some time to complete.

----

### Get Dubbed Run Info

You can use this method to get the links for downloading the dubbed video and audio,<br>
as well as the transcript of the dubbed video in the target language.

In [ ]:
dubbed_info = client.get_dubbed_run_info(run_id)

In [ ]:
dubbed_info["video_url"]

In [ ]:
dubbed_info["audio_url"]

In [ ]:
dubbed_info["transcript"]

----

To know more, check out Camb AI's [API Documentation](https://docs.camb.ai).